In [43]:
import math
import pyFAI
from pyFAI.geometry import Geometry
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
import matplotlib.pyplot as plt
import numpy as np
from tifffile.tifffile import imread
import os
import scipy.stats as sts
# Define some functions for masking
def margin(img_shape, edge_size):
    """
    Mask the edge of an image

    Parameters
    -----------
    img_shape: tuple
        The shape of the image
    edge_size: int
        Number of pixels to mask from the edge

    Returns
    --------
    2darray:
        The mask array, bad pixels are 0
    """
    mask = np.ones(img_shape, dtype=bool)
    mask[edge_size:-edge_size, edge_size:-edge_size] = 0.
    return ~mask


def binned_outlier(img, r, alpha, bins, mask=None):
    """
    Generates a mask by identifying outlier pixels in bins and masks any
    pixels which have a value greater or less than alpha * std away from the
    mean

    Parameters
    ----------
    img: 2darray
        The  image
    r: 2darray
        The  array which maps pixels to bins
    alpha: float or tuple or, 1darray
        Then number of acceptable standard deviations, if tuple then we use
        a linear distribution of alphas from alpha[0] to alpha[1], if array
        then we just use that as the distribution of alphas
    bins: list
        The bin edges
    mask: 1darray
        A starting flattened mask

    Returns
    --------
    2darray:
        The mask
    """

    if mask is None:
        wroking_mask = np.ones_like(img.shape).astype(bool)
    else:
        working_mask = mask.copy()
    if working_mask.shape != img.shape:
        working_mask = working_mask.reshape(img.shape)
    msk_img = img[working_mask]
    msk_r = r[working_mask]

    int_r = np.digitize(r, bins[:-1], True) - 1
    # integration
    mean = sts.binned_statistic(msk_r, msk_img, bins=bins,
                                statistic='mean')[0]
    std = sts.binned_statistic(msk_r, msk_img, bins=bins,
                               statistic=np.std)[0]
    if type(alpha) is tuple:
        alpha = np.linspace(alpha[0], alpha[1], len(std))
    threshold = alpha * std
    lower = mean - threshold
    upper = mean + threshold

    # single out the too low and too high pixels
    working_mask *= img > lower[int_r]
    working_mask *= img < upper[int_r]

    return working_mask.astype(bool)


def bs_mask(shape, center, width):
    bs = np.ones(shape, dtype=bool)
    bs[0:center[0], center[1] - width:center[1] + width] = 0.
    return bs


from skbeam.io.save_powder_output import _create_file_path

header = b"M\x00\x00\x00A\x00\x00\x00S\x00\x00\x00K\x00\x00\x00\x00\x08" \
             b"\x00\x00\x00\x08\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
             b"\x00"


def fit2d_save(mask, filename, dir_path=None):
    """
    Compresses and wraps the mask for Fit2D use
    Parameters
    ----------
    mask: ndarray
        The mask
    filename: str
        The filename
    dir_path: str, optional
        Path to the destination file
    """
    # Fit2d works with inverted masks
    mask = np.flipud(mask)
    mask = np.abs(mask - 1)
    b1, b2 = mask.shape
    b28 = b2 // 8
    c = np.zeros((b1, b28), dtype=np.uint8)
    # bitmap compression for Fit2D io
    # TODO: possibly parallizable
    for i in range(0, b1):
        j = 0
        while j < b2:
            c[i, math.floor(j / 8)] = mask[i, j]
            for k in range(1, 8):
                c[i, math.floor(j / 8)] += mask[i, j + k] * (2 ** k)
            j += 8
    maskname = _create_file_path(dir_path, filename, '.msk')
    with open(maskname, "wb") as fout:
        c.tofile(fout, "", "%x")
    # Giant long mask header, must be there for fit2d to work with the mask
    with open(maskname, 'rb') as mask:
        bodypt1 = mask.read(262144)
        bodypt2 = mask.read()

    body = [bodypt1, bodypt2]
    body2 = b"".join(body)
    total = [header, body2]
    total2 = b"".join(total)
    with open(maskname, "wb") as fout:
        fout.write(total2)

In [45]:
# load the pyFAI stuff
geo = Geometry()
# Preferably from pyFAI calibration?
# geo = pyFAI.load('filename')
geo.setFit2D(
    directDist=25,
    tilt=.5,
    tiltPlanRotation=.5,
    centerX=1024,
    centerY=1024,
    pixelX=200,
    pixelY=200,
)
# put your image path in here
path = os.path.expanduser('~/shared/PDFdata/July/X17A/1/FinalSum_1.tif')
tmsk = None
lower_thresh = 0.0
upper_thresh = None
edge_size = 30
bs_size = 20
alpha = 4

Detector Detector	 Spline= None	 PixelSize= 2.000e-04, 2.000e-04 m
SampleDetDist= 2.499905e-02m	PONI= 2.047981e-01, 2.045818e-01m	rot1=-0.008726  rot2= 0.000076  rot3= -0.000000 rad
DirectBeamDist= 25.000mm	Center: x=1024.000, y=1024.000 pix	Tilt=0.500 deg  tiltPlanRotation= 0.500 deg

In [46]:
# make the radial array
r = geo.rArray((2048, 2048))
pixel_size = [getattr(geo, a) for a in ['pixel1', 'pixel2']]
rres = np.hypot(*pixel_size)
rbins = np.arange(np.min(r) - rres / 2., np.max(r) + rres / 2., rres)
img = tifffile.imread(path)
img = np.flipud(img)

In [49]:
if tmsk is None:
    tmsk = np.ones(img.shape, dtype=int).astype(bool)
if margin:
    tmsk *= margin(img.shape, edge_size)
if lower_thresh is not None:
    tmsk *= (img > lower_thresh).astype(bool)
if upper_thresh:
    tmsk *= (img < upper_thresh).astype(bool)
if bs_size:
    center = np.unravel_index(np.argmin(r), img.shape)
    bsm = bs_mask(img.shape, center,
                    bs_size)
    a = chi[(center[0] - 20, center[1] - bs_size)]
    b = chi[(center[0] - 20, center[1] + bs_size)]
    criteria = (a < chi) & (chi < b)
    bsm[~criteria] = True
    tmsk *= bsm
if alpha:
    tmsk *= binned_outlier(img, r, alpha, rbins, mask=tmsk)

/home/christopher/mc/envs/dp/lib/python3.5/site-packages/ipykernel/__main__.py:67: RuntimeWarning: invalid value encountered in less
/home/christopher/mc/envs/dp/lib/python3.5/site-packages/ipykernel/__main__.py:68: RuntimeWarning: invalid value encountered in greater


In [ ]:
masked_img = img.copy()
masked_img[np.where(~tmsk)] = np.nan
fig, axs = plt.subplots(1, 3, sharex=True, sharey=True)
for ax, data in zip(axs, [img, tmsk, masked_img]):
    ax.imshow(data, cmap='viridis')
plt.show()

In [ ]:
fit2d_save(tmsk, 'maskname.msk')